In [1]:
#Here to
import numpy as np
import pandas as pd
import scipy.stats as stats

import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16})
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['grid.color'] = 'lightgrey'
# docker start sparkbook
import pyspark as ps

spark = (ps.sql.SparkSession.builder 
        .master("local[6]") 
        .appName("case study") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [3]:
import nltk
from nltk.corpus import stopwords
en_stop = ['lbc','amp','au', 'aux', 'avec', 'ca' 'ce', 'ces', 'cest', 'dans', 'de', 'des', 
           'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 
           'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 
           'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 
           'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 
           'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 
           'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 
           'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 
           'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 
           'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 
           'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 
           'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 
           'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 
           'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 
           'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 
           'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 
           'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 
           'aurez', 'auront', 'aurais', 'aurait', 'aurions', 'auriez', 
           'auraient', 'avais', 'avait', 'avions', 'aviez', 'avaient', 
           'eut', 'eûmes', 'eûtes', 'eurent', 'aie', 'aies', 'ait', 
           'ayons', 'ayez', 'aient', 'eusse', 'eusses', 'eût', 'eussions',
           'eussiez', 'eussent','i', 'me', 'my', 'myself', 'we', 'our', 
           'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
           "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 
           'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', 
           "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 
           'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 
           "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 
           'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 
           'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 
           'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
           'with', 'about', 'against', 'between', 'into', 'through', 'during',
           'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
           'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
           'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 
           'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 
           'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 
           't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now',
           'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', 
           "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 
           'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', 
           "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 
           'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
           "won't", 'wouldn', "wouldn't", 'si']

In [4]:
df_json = spark.read.json('../data/french_tweets.json')
#df2_json = spark.read.json('/home/jovyan/work/Galva/caseStudy/Spark/spark-case-study/peer_extension/tweets.20150430-223406.json')
# create a temporary table for spark.sql queries
df_json.createOrReplaceTempView('temp_view')


In [46]:
df_json2.createOrReplaceTempView('temp2')

NameError: name 'df_json2' is not defined

In [33]:
nlp_sql1 = spark.sql('''
SELECT lang , text,created_at, possibly_sensitive, quoted_status.favorite_count
FROM temp_view
WHERE lang = 'en' and possibly_sensitive = "True";
''').collect()

In [34]:
nlp_sql2 = spark.sql('''
SELECT lang , text,created_at, possibly_sensitive, quoted_status.retweet_count
FROM temp_view
WHERE lang = 'en' and possibly_sensitive = "False";
''').collect()

In [6]:
#try and search/identify for Porn type text among tweets
#among sensitive tweets with lots of re_tweets or likes_ examine more closely 
###perhaps deep dive among some volume of like_counts or re_tweet count top 5%? 
nlp_sql1[:3]

[Row(lang='en', text='The Old Blind Guitarist by Pablo Picasso https://t.co/qNxPnN3jI7', created_at='Wed Apr 26 13:34:36 +0000 2017', possibly_sensitive=True, favorite_count=None),
 Row(lang='en', text='Futur achat 😘 https://t.co/DASepOFASa', created_at='Wed Apr 26 13:34:42 +0000 2017', possibly_sensitive=True, favorite_count=23),
 Row(lang='en', text='SCH - Comme si https://t.co/vaPuC4bxYp', created_at='Wed Apr 26 14:42:10 +0000 2017', possibly_sensitive=True, favorite_count=None)]

In [7]:
#Create a function to filter symbols from nlp_sql1

In [5]:
def clean_text(row):
    to_remove = '''!_~^(){}@:'$%"\,-[]<>./?;@#&*'''''
     
    return ''.join([item for item in row if item not in to_remove]) 

In [9]:
# def get_counts(sql_results):
#     '''
#     input = spark list_of_rows
#     does = cleaning of string
#     output= favorites,string
#     '''
#     countz = {}
#     for index,item in enumerate(sql_results):
#         clean =clean_text(item[1])
#         #print(clean)
#         favs = str(item[4])
#         countz[index] = (favs,clean)
#     #now have a dict of likes/tweet_text
#     return countz

# countz = get_counts(sql_results = nlp_sql1)


In [10]:
#countz2 = get_counts(sql_results = nlp_sql2)

In [11]:
#print(countz[109]) -- > ('None', 'new Blackedcom trailer with the gorgeous MBlueEyesXxX \n
                                    # please RT and follow me and Markenna httpstcoNGWSjEZap4')
# def clean_http():
#     lst = []
#     for k,v in countz2.items(): #countz or countz2
#         v_ = v[1].split(' ')
#         del v_[-1]
#         for item in v_:
#             lst.append(item)
#     return lst
# no_http2 = clean_http() 

In [12]:
# print (no_http[2:10]) --> ['Blind', 'Guitarist', 'by', 'Pablo', 'Picasso', 'Futur', 'achat', '😘']
#still not able to remove emojies

In [13]:
# Inputs for below could could be as follows
# 202: ('None', 'so i went to a party tonight amp asked for the new shawns music and they played stitches WELL httpstcoWHoGPRcv3s'), 
# 203: ('50', '5 millions  httpstcoJGyUNbunZV')

def somefunct(x):
    word_dict = {}
    counter =0

    for k,v in x.items():
        counter += 1
        likez = v[0] 
        v=list(v[1].split(' ')) #convert text in list of words
        #example_output for (v) --> ['The', 'Old', 'Blind', 'Guitarist', 'by', 'Pablo', 'Picasso', 'httpstcoqNxPnN3jI7']

        for item in v: # for each word in the list 
            v_ = [item]
            boozl =''
            boozl = likez
            if boozl == 'None': #workaround for dealing with None strings
                likez = 'Nope' 
            if item not in word_dict:  
                if boozl != 'Nope':
                    word_dict[item] = list([boozl])
            else:
                if boozl != 'Nope':
                    word_dict[item].append( boozl )
    return word_dict

somefunct_output = somefunct(countz2)  #countz or countz2

# acum=0
# for item in somefunct_output.values():
#     unravel = len([x for x in item])
#     acum+=unravel
# print(acum) --> 602 #Count of words <with overlap however 4/26 --> or someother time remove overlap ?

NameError: name 'countz2' is not defined

In [ ]:
stringz = ' '.join([i for i in no_http])

In [ ]:
stringz2 = ' '.join([i for i in no_http2])

In [ ]:
print(len('😘'))
import demoji
demoji.download_codes()

In [6]:
import re
import sys

# https://stackoverflow.com/a/49146722/330558
def remove_emoji(x):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', x)

 
    
text2 = remove_emoji(x) # either stringz or stringz

NameError: name 'x' is not defined

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
wordcloud = WordCloud(stopwords=en_stop , 
                      width=1200, 
                      height=800, 
                      min_word_length=3,
                      max_words=15).generate(text2)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.savefig('enFAL_cloud_top15.png')
plt.show()

In [7]:
def clean_http2(x):
    lst = []
    x_ = x.lower().split(' ')
    del x_[-1]
    return x_
no_http22 = clean_http2(x)
print(no_http22)

NameError: name 'x' is not defined

In [8]:
q2 = spark.sql('''
SELECT   text 
FROM temp_view
WHERE lang = 'en' and possibly_sensitive = "True";
''').collect()

In [9]:
q3 = spark.sql('''
SELECT   text 
FROM temp_view
WHERE lang = 'en' and possibly_sensitive = "False";
''').collect()

In [19]:
len(q3)

20020

In [22]:
qw = spark.sql('''
SELECT   text 
FROM temp_view
WHERE lang='en';
''').collect()

In [23]:
len(qw)

41831

In [39]:
# Create a function to process each row 
q2[0:10]
#Clean each row
##lowercase i.lower(), remove emojies remove_emojies() , remove http with clearn_http() , clean with clean_text() 

[Row(text='The Old Blind Guitarist by Pablo Picasso https://t.co/qNxPnN3jI7'),
 Row(text='Futur achat 😘 https://t.co/DASepOFASa'),
 Row(text='SCH - Comme si https://t.co/vaPuC4bxYp'),
 Row(text='new @PornDoePremium trailer with two perfect girls @Anna_swix @NaomiBenneta\nplease RT and follow me Anna and Naomi https://t.co/1niJXq1W1f'),
 Row(text='Sch- Poupee Russe (clip offeciel ) https://t.co/V0cEuRKtx7'),
 Row(text="imagine being this jealous, can't relate https://t.co/fTvOuONvhP"),
 Row(text='@MisGrace @LBC no news on this, lbc is a disgrace https://t.co/ytdplU1aYS …'),
 Row(text='@Marsi132 @LBC @AlexSalmond @IainDale lbc no news = https://t.co/ytdplU1aYS …'),
 Row(text='@_SEE_FUTURE @JonathanFrakes @gossippsychic @SkyNews @BBCNews @LBC did she see - Facts About 9/11 | Global Research… https://t.co/7mzPtAgovT'),
 Row(text='@chunkymark @MichaelLee2009 Facts About 9/11 | Global Research - https://t.co/NwtNaVDlGF')]

In [10]:
def clean2(sql_results):
    '''
    input = spark list_of_rows
    does = cleaning of string
    output= favorites,string
    '''
    countz_d = {}
    for index,item in enumerate(sql_results):
#         if index == 10:
#             return countz_d 
        clean =clean_text(item) # prehaps cast clean to a list,
        #clean2 = clean_text(clean) # using clean2 results in keeping screen names, which could be topics?
        remove_emoj = remove_emoji(clean)
        remove_http = clean_http2(remove_emoj)
        remove_http_ =[item for item in remove_http if item != '@']
        countz_d[index]=remove_http_ 
        
    return countz_d  

In [15]:
cleaned = clean2(sql_results = q2)

In [12]:
cleaned2 =clean2(sql_results = q3)

d3={}
for kk,vv in cleaned2.items():
    lst2=[]
    for wword in vv:
        ww = list(wword) 
         
        if len(ww) > 1 and ww[0] != '@':
            v_ = clean_text(wword)
            lst2.append(v_)
        d3[kk]=lst2
print(len(d3))

20015


In [16]:
dd={}
for k,v in cleaned.items():
    lst=[]
    for word in v:
        w = list(word) 
         
        if len(w) > 1 and w[0] != '@':
            v_ = clean_text(word)
            lst.append(v_)
        dd[k]=lst
print(len(dd))

204


In [33]:
# here to LDA is pyspark walkthrough
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA

In [34]:
dmx = spark.createDataFrame([[1, Vectors.dense([0.0, 1.0])],[2, SparseVector(2, {0: 1.0})],], ["id", "features"])

In [37]:
dmx.show()

+---+-------------+
| id|     features|
+---+-------------+
|  1|    [0.0,1.0]|
|  2|(2,[0],[1.0])|
+---+-------------+



In [38]:
lda = LDA(k=2, seed=1, optimizer="em")

In [40]:
lda.setMaxIter(10)

LDA_403808e1344e

In [43]:
dd_str = ''
acum=4
while acum <5:
    for item in dd.values():
        item_ = ' '.join(item)
        acum+=1
        dd_str +=item_ + ' '
print(len(dd_str))

9238


Web Scrapping Tweets

In [ ]:
spark.stop()

In [34]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
vectorizer = CountVectorizer()
x= vectorizer.fit_transform(dd_str)

ValueError: Iterable over raw text documents expected, string object received.

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ['This is the first document.','This document is the second document.','And this is the third one.','Is this the first document?']

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())


['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [ ]:
'''
5/3/21

Trying to get LDA working -  
using this for the TFIDF part https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html 
because the TFIDF methods from the LDA lecture is not working and I cant make sense of the all digit example in pyspark

THEN can continue on within the prep data for LDA 

'''